In [3]:
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np

from GEMS_TCO import data_preprocess 
from GEMS_TCO import  kernels_reparam_space_time as kernels_reparam_space_time
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2, exact_location_filter
import torch
import torch.optim as optim
import time

from GEMS_TCO import debiased_whittle

Load monthly data

In [4]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 20
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)


df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      
lon_range=[123.0, 133.0] 
)

Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


Load daily data applying max-min ordering

In [5]:
daily_aggregated_tensors = [] 
daily_hourly_maps = []      

daily_aggregated_tensors2 = [] 
daily_hourly_maps2 = []    

for day_index in range(31):
    hour_start_index = day_index * 8
    hour_end_index = (day_index + 1) * 8
    #hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]
    
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    hour_indices, 
    ord_mm= None,  # or just omit it
    dtype=torch.float, # or just omit it 
    keep_ori=False
    )

    daily_aggregated_tensors.append( day_aggregated_tensor )
    daily_hourly_maps.append( day_hourly_map )


    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    hour_indices, 
    ord_mm= None,  # or just omit it
    dtype=torch.float, # or just omit it 
    keep_ori=True
    )

    daily_aggregated_tensors2.append( day_aggregated_tensor )
    daily_hourly_maps2.append( day_hourly_map )


print(daily_aggregated_tensors[0].shape)
print(daily_aggregated_tensors[0][:10])

torch.Size([145008, 4])
tensor([[  4.9760, 132.9840, 260.9815,  21.0000],
        [  4.9760, 132.9210, 261.5255,  21.0000],
        [  4.9760, 132.8580, 262.9575,  21.0000],
        [  4.9760, 132.7950, 262.6640,  21.0000],
        [  4.9760, 132.7320, 261.0027,  21.0000],
        [  4.9760, 132.6690, 260.6678,  21.0000],
        [  4.9760, 132.6060, 261.8147,  21.0000],
        [  4.9760, 132.5430, 263.7160,  21.0000],
        [  4.9760, 132.4800, 263.3271,  21.0000],
        [  4.9760, 132.4170, 263.4973,  21.0000]])


In [28]:
lat_d = 0.9
lon_d = 0.4

daily_hourly_filtered_maps, daily_aggregated_filtered_maps = exact_location_filter.filter_by_location_deviation(daily_hourly_maps, daily_hourly_maps2, daily_aggregated_tensors, daily_aggregated_tensors2, lat_d, lon_d)

#ord_mm, nns_map = exact_location_filter.get_spatial_ordering(daily_hourly_filtered_maps[0], 10)
#print(nns_map)

--- Starting Deviation Filter (Days 0-30) ---
Thresholds: Lat <= 0.9, Lon <= 0.4
Day 0 (Hourly): 18126 -> 16372 rows kept (90.32%)
Day 0 (Aggregated): 145008 -> 130976 rows kept (Shape consistent with 16372 filtered hourly rows).
------------------------------
Day 1 (Hourly): 18126 -> 16391 rows kept (90.43%)
Day 1 (Aggregated): 145008 -> 131128 rows kept (Shape consistent with 16391 filtered hourly rows).
------------------------------
Day 2 (Hourly): 18126 -> 16389 rows kept (90.42%)
Day 2 (Aggregated): 145008 -> 131112 rows kept (Shape consistent with 16389 filtered hourly rows).
------------------------------
Day 3 (Hourly): 18126 -> 16392 rows kept (90.43%)
Day 3 (Aggregated): 145008 -> 131136 rows kept (Shape consistent with 16392 filtered hourly rows).
------------------------------
Day 4 (Hourly): 18126 -> 16389 rows kept (90.42%)
Day 4 (Aggregated): 145008 -> 131112 rows kept (Shape consistent with 16389 filtered hourly rows).
------------------------------
Day 5 (Hourly): 181

In [13]:
v = 0.5 # smooth
mm_cond_number = 8
nheads = 300
#nheads = 1230
#lr = 0.01
#step = 80
#gamma_par = 0.5

# --- Placeholder Global Variables ---
# 💥 REVISED: Added lr, patience, factor. Removed step, gamma_par
lr=0.1
patience = 5       # Scheduler: Epochs to wait for improvement
factor = 0.5         # Scheduler: Factor to reduce LR by (e.g., 0.5 = 50% cut)
epochs=100

In [14]:
# --- L-BFGS SPECIFIC GLOBAL PARAMETERS ---
LBFGS_LR = 1.0
LBFGS_MAX_STEPS = 10       # Number of outer optimization steps
LBFGS_HISTORY_SIZE = 100   # Memory for Hessian approximation
LBFGS_MAX_EVAL = 50        # Max evaluations (line search) per step

# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [1] # 0 index 
for day_idx in day_indices:  

    # Assuming data access is correct
    daily_hourly_map = daily_hourly_filtered_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_filtered_maps[day_idx]

    ord_mm, nns_map = exact_location_filter.get_spatial_ordering(daily_hourly_filtered_maps[0], mm_cond_number)

    for map in daily_hourly_map:
        daily_hourly_map[map] = daily_hourly_map[map][ord_mm]

    # --- Parameter Initialization (SPATIO-TEMPORAL) ---
    '''  
    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    init_range_time = 0.1
    init_advec_lat = 0.02
    init_advec_lon = -0.08
    '''
    init_sigmasq   = 13.059
    init_range_lat = 0.154 
    init_range_lon = 0.195 
    init_nugget    = 0.247
    init_range_time = 0.7
    init_advec_lat = 0.0218
    init_advec_lon = -0.1689

    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # 1/range_lon
    init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
    init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 7-parameter spatio-temporal list (Log/Linear)
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [0] log(phi1)
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [1] log(phi2)
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [2] log(phi3)
        torch.tensor([np.log(init_phi4)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [3] log(phi4)
        torch.tensor([init_advec_lat],         requires_grad=True, dtype=torch.float64, device=device_str ), # [4] advec_lat (linear)
        torch.tensor([init_advec_lon],         requires_grad=True, dtype=torch.float64, device=device_str ), # [5] advec_lon (linear)
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )  # [6] log(nugget)
    ]

    # --- Define parameter groups ---
    lr_all = LBFGS_LR
    all_indices = [0, 1, 2, 3, 4, 5, 6] 
    
    # L-BFGS requires the parameters to be iterable in a single list or group
    param_groups = [
        {'params': [params_list[idx] for idx in all_indices], 'lr': lr_all, 'name': 'all_params'}
    ]

    # --- Print Job Info (using placeholder print variables) ---
    res_calc = (113 // lat_lon_resolution[0]) * (158 // lat_lon_resolution[0]) 
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { daily_aggregated_tensor.shape[0]/8}, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n')
    for i, p in enumerate(params_list):
        print(f"  Param {i}: {p.item():.4f}")
            
    # --- 💥 Instantiate the L-BFGS Class ---
    # NOTE: Assuming fit_vecchia_lbfgs is available via kernels_reparam_space_time
    model_instance = kernels_reparam_space_time.fit_vecchia_lbfgs(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- 💥 Set L-BFGS Optimizer ---
    # L-BFGS specific arguments are passed here
    optimizer = model_instance.set_optimizer(
            param_groups,     
            lr=LBFGS_LR,            
            max_iter=LBFGS_MAX_EVAL,        # max_iter in LBFGS is the line search limit
            history_size=LBFGS_HISTORY_SIZE 
        )

    # --- 💥 Call the L-BFGS Fit Method ---
    out, steps_ran = model_instance.fit_vecc_lbfgs(
            params_list,
            optimizer,
            model_instance.matern_cov_aniso_STABLE_log_reparam, 
            max_steps=LBFGS_MAX_STEPS # Outer loop steps
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {steps_ran+1} L-BFGS steps.")
    print(f"Day {day_idx+1} final results (raw params + loss): {out}")
# day 0 61m 


--- Starting Day 2 (2024-07-2) ---
Data size per day: 15181.0, smooth: 0.5
mm_cond_number: 8,
initial parameters: 

  Param 0: 4.2042
  Param 1: 1.6348
  Param 2: 0.4721
  Param 3: -2.5562
  Param 4: 0.0218
  Param 5: -0.1689
  Param 6: -1.3984
--- Starting L-BFGS Optimization ---
--- Step 1/10 / Loss: 1.490161 ---
  Param 0: Value=3.3634, Grad=1.3591195052272828e-06
  Param 1: Value=1.4000, Grad=4.953914267371197e-07
  Param 2: Value=-1.5226, Grad=1.8429332092548725e-08
  Param 3: Value=-3.1225, Grad=1.4741271117473528e-07
  Param 4: Value=0.1083, Grad=1.400311751110981e-07
  Param 5: Value=-0.2207, Grad=3.759825409581295e-06
  Param 6: Value=1.1187, Grad=2.8386796741426228e-06
  Max Abs Grad: 3.759825e-06
------------------------------

Converged on gradient norm (max|grad| < 1e-05) at step 1
FINAL STATE: Step 1, Loss: 1.490160846159536
  Raw (vecc) Parameters: [3.36336504695141, 1.4000489357246877, -1.5226391671330102, -3.122542129387705, 0.10834442729589913, -0.22068132726903175, 

full nll

In [10]:
ord_mm, nns_map = exact_location_filter.get_spatial_ordering(daily_hourly_filtered_maps[0], 10)
print(nns_map)

[[   -1    -1    -1 ...    -1    -1    -1]
 [    0    -1    -1 ...    -1    -1    -1]
 [    0     1    -1 ...    -1    -1    -1]
 ...
 [ 2309   129  3182 ...  6499 12043  3169]
 [ 2267   947  3193 ... 11176 11662  2524]
 [  122  2445  3181 ... 11779 11978 12045]]


In [6]:
nn = daily_aggregated_tensors[0].shape[0]
print(nn)
day_idx = 0
lat_range= [1,3]
lon_range= [125.0, 129.0]

day2_va = [4.263671207178608, 1.6401967914088247, 0.25443837667152014, -4.020499340539446, 0.026886942200999114, -0.16593421415829276, -0.6337447467203996]
day2_vl = [4.363671207178608, 1.6401967914088247, 0.25443837667152014, -4.020499340539446, 0.026886942200999114, -0.16593421415829276, -0.6337447467203996]
day2_dwl = [4.163671207178608, 1.6401967914088247, 0.25443837667152014, -4.020499340539446, 0.026886942200999114, -0.16593421415829276, -0.6337447467203996]

day = [ day2_va, day2_vl, day2_dwl]

for i,model_params in enumerate(day):
    instance = debiased_whittle.full_vecc_dw_likelihoods(daily_aggregated_tensors, daily_hourly_maps, day_idx=day_idx, params_list=model_params, lat_range=lat_range, lon_range=lon_range)
    v = 0.5
    nheads = 300
    instance.initiate_model_instance_vecchia(v, nns_map, mm_cond_number, nheads)
    res = instance.likelihood_wrapper()
    print(f' full likelihood: {torch.round(res[0]*nn, decimals=2)},\n vecchia: {torch.round(res[1]*nn, decimals=2)}, \n whittle de-biased: {torch.round(res[2], decimals = 2)}')

145008


: 